In [100]:
!pip install -U spacy

import numpy as np
import pandas as pd
from string import punctuation
import nltk
import spacy
from spacy.lang.pt import Portuguese
nltk.download('stopwords')
nltk.download('rslp')
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import re
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leosilva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /Users/leosilva/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [77]:
oplexicon = open('/Users/leosilva/development/thesis/Lexicos/oplexicon_v3.0/lexico_v3.0.txt','r')

In [78]:
nlp = Portuguese()

In [79]:
data = pd.read_json(r'/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/alldata.json')
data.head()

,26,27,28,29,30,31
id_str_twitter,23781146,15148309,927028057763995648,77648005,3429765093,10794662
name,Fe Rebelatto,Sommelier de Airbnb,✪ Felippe,Conference Distinguished Specialist,Paula Santana,Maharaja of the Legacies
screen_name,rebelatto,Guilh_rm_,FelippeRegazio,NannoKa,psanrosa13,rponte
location,"Rio de Janeiro, Brasil",somewhere in Brazil,,Ctba-PR Brasil,Brasil,"Ceará, Brazil"
url,None,https://t.co/gX4oRZWHg3,https://t.co/KkaPlVH4Wu,https://t.co/hDni4Fh2Ng,https://t.co/YJQuDtv2YW,https://t.co/3yZ94QNQvs


In [80]:
dic_palavra_polaridade = {}

for i in oplexicon.readlines():
    linha_splitted = i.split(',')
    palavra = linha_splitted[0]
    polaridade = linha_splitted[2]
    dic_palavra_polaridade['{}'.format(palavra)] = polaridade

print(len(dic_palavra_polaridade))


32119


In [81]:
def score_sentimento(frase):
    frase = frase.lower()
    l_sentimento = []
    for p in frase.split():
        l_sentimento.append(int(dic_palavra_polaridade.get(p, 0)))
    score = sum(l_sentimento)
    return score

In [82]:
score_sentimento('Estou Muito Feliz hoje,super animado com o trabalho novo! :)')

3

In [83]:
dic_tweets_per_user = {}

for c in data:
    id_user = c
    tweets = []
    for t in data[id_user]['tweets']:
        tweets.append({"id": t[0], "id_str_twitter": t[1], "text": t[2], "oplexicon": None})

        dic_tweets_per_user['{}'.format(id_user)] = tweets

In [101]:
# cleaning the tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [117]:
def clean_data(tweet_text):
    # remove twitter Return handles (RT @xxx:)
    tweet_text = np.vectorize(remove_pattern)(tweet_text, "RT @[\w]*:")

    # remove twitter handles (@xxx)
    tweet_text = np.vectorize(remove_pattern)(tweet_text, "@[\w]*")
    
#     tweet_text = re.sub(r"http\S+", "", tweet_text).lower()
#     tweet_text = re.sub(r"[-|0-9]", "", tweet_text).lower()
#     tweet_text = re.sub(r'[-./?!,":;()\']', ' ', tweet_text).lower()
    
    # colocar para funcionar o seguinte algoritmo:
    
    # remove URL links (httpxxx)
    tweet_text = np.vectorize(remove_pattern)(tweet_text, "https?://[A-Za-z0-9./]*")

    # remove special characters, numbers, punctuations (except for #)
    tweet_text = np.core.defchararray.replace(tweet_text, "[^a-zA-Z]", " ")
    
    tweet_text = np.array2string(tweet_text)
    
    return tweet_text

In [85]:
stop_words = stopwords.words('portuguese')

pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

def remove_stop_words(tweet_text): # Função para remover stopwords e pontuações
    tweet_text = nlp(tweet_text)
    palavras = [palavra.text for palavra in tweet_text if palavra.text not in stop_words and palavra.text not in pontuacao]
    return " ".join(palavras)

In [86]:
def remove_repeated_letters(tweet_text): # Função para remover letras repetidas
    tweet_text = nlp(tweet_text)
    palavras = list()
    for palavra in tweet_text:
        if palavra.text not in list(dic_palavra_polaridade.keys()):
            palavras.append(re.compile(r'(.)\1{1,}', re.IGNORECASE).sub(r'\1', palavra.text))
        else:
            palavras.append(palavra.text)
    return (" ".join(palavras))

In [87]:
def stemming(tweet_text):
    stemmer = RSLPStemmer()
    tweet_splitted = tweet_text.split(' ')
    phrase = []
    for word in tweet_splitted:
        if word is not '':
            phrase.append(stemmer.stem(word.lower()))
    return (" ".join(phrase))

In [96]:
def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    return norm_score

In [119]:
pos_scores = []
norm_pos_scores = []
neg_scores = []
norm_neg_scores = []
neu_scores = []

for u in dic_tweets_per_user:
    tweets = dic_tweets_per_user[u]
    for t in tweets:
        tweet = t['text']
        tweet = clean_data(tweet)
        tweet = remove_stop_words(tweet)
        tweet = remove_repeated_letters(tweet)
        #tweet = stemming(tweet)
        oplexicon_analysis = score_sentimento(tweet)
        if oplexicon_analysis > 0:
            pos_scores.append(oplexicon_analysis)
            oplexicon_analysis = round(normalize(oplexicon_analysis))
            norm_pos_scores.append(oplexicon_analysis)
        elif oplexicon_analysis == 0:
            neu_scores.append(oplexicon_analysis)
        else:
            neg_scores.append(oplexicon_analysis)
            oplexicon_analysis = round(normalize(oplexicon_analysis))
            norm_neg_scores.append(oplexicon_analysis)
            
print('Positivos: {}'.format(len(pos_scores)))
print('Negativos: {}'.format(len(neg_scores)))
print('Neutros: {}'.format(len(neu_scores)))

print('\n')
print('Pos Score: {}'.format(pos_scores[0]))
print('Norm Pos Score: {}'.format(norm_pos_scores[0]))
print('Pos Score: {}'.format(pos_scores[1]))
print('Norm Pos Score: {}'.format(norm_pos_scores[1]))
print('Pos Score: {}'.format(pos_scores[2]))
print('Norm Pos Score: {}'.format(norm_pos_scores[2]))
print('Pos Score: {}'.format(pos_scores[3]))
print('Norm Pos Score: {}'.format(norm_pos_scores[3]))
print('Pos Score: {}'.format(pos_scores[4]))
print('Norm Pos Score: {}'.format(norm_pos_scores[4]))

print('\n')
print('Neg Score: {}'.format(neg_scores[0]))
print('Norm Neg Score: {}'.format(norm_neg_scores[0]))
print('Neg Score: {}'.format(neg_scores[1]))
print('Norm Neg Score: {}'.format(norm_neg_scores[1]))
print('Neg Score: {}'.format(neg_scores[2]))
print('Norm Neg Score: {}'.format(norm_neg_scores[2]))
print('Neg Score: {}'.format(neg_scores[3]))
print('Norm Neg Score: {}'.format(norm_neg_scores[3]))
print('Neg Score: {}'.format(neg_scores[4]))
print('Norm Neg Score: {}'.format(norm_neg_scores[4]))

Positivos: 9377
Negativos: 5253
Neutros: 10976


Pos Score: 2
Norm Pos Score: 0
Pos Score: 2
Norm Pos Score: 0
Pos Score: 2
Norm Pos Score: 0
Pos Score: 1
Norm Pos Score: 0
Pos Score: 1
Norm Pos Score: 0


Neg Score: -1
Norm Neg Score: 0
Neg Score: -1
Norm Neg Score: 0
Neg Score: -1
Norm Neg Score: 0
Neg Score: -2
Norm Neg Score: 0
Neg Score: -3
Norm Neg Score: -1
